# Introduction

All student names in group: Dong Liang, Kevin Zheng, Hojat Jaffary

I understand that my submission needs to be my own group's work: DL, KZ, HJ

I understand that ChatGPT / Copilot / other AI tools are not allowed: DL, KZ, HJ

Points: 10

Complete this notebook and submit it (save/print as pdf). Make sure all output is correct in the pdf before submitting (it sometimes gets cut off). The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [ ]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 9.3 MB/s eta 0:00:00


In [ ]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(1 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(1 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(1 point)**

Solution (Assuming 1-indexed):


**Number 5 can occur at most once in the first row**

- i must be in row 1. j is column 1 through 9. k must be 5, and in the formula we will get each distinct pair and both cannot be true as it will make the constraint false. At least one must be false
- ~($X_{1,j}^5$ AND $X_{1,z}^5$), 1 $\leq$ j $\le$ z $\leq$ 9 (all distinct pairs while j and z will not be equal)


**Number 6 can occur at most once in the first column**

- j must be in column 1. i is row 1 through 9. k must be 6, and in the formula we will get each distinct pair and both cannot be true as it will make the constraint false. At least one must be false
- ~($X_{i,1}^6$ AND $X_{z,1}^6$), 1 $\leq$ i $\le$ z $\leq$ 9 (all distinct pairs while j and z will not be equal)


**Number 9 can occur at most once in the top left 3 × 3 sub-grid.**

- i is row 1 through 3. j is column 1 through 3. k must be 9, an din the formula we will get each distinct pair of the entire 3 x 3 and make sure that there must be at least one false when comparing each of the positions.
- ~($X_{i,j}^9$ AND $X_{k,l}^9$), 1 $\leq$ i, j, k, l $\leq$ 3. (i,j) $\neq$ (k,l). Each pair in the 3x3 can not contain more than one 9


## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(1 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [ ]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

s = Solver()

# All possible combination of 9x9 matrix of Bool variables (row i, col j, value k-1) - 0-indexed
X = [[[
    Bool("cell_%s_%s_%s"%(i, j, k))
    for k in range(1, 10)]
    for j in range(9)]
    for i in range(9)]

# Constraint: 1 number per cell
valid_cell = [
   Sum([If(X[i][j][k], 1, 0) for k in range(9)]) == 1
   for j in range(9)
   for i in range(9)
]

# Constraint: Distinct rows
distinct_row = [
    Not(And(X[i][c1][k], X[i][c2][k]))
    for c1 in range(9)
    for c2 in range(9) if(c2 != c1) # Don't check itself with itself
    for i in range(9)
    for k in range(9)
]


# Constraint: Distinct cols
distinct_col = [
    Not(And(X[r1][j][k], X[r2][j][k]))
    for r1 in range(9)
    for r2 in range(9) if (r2 != r1) # Don't check itself with itself
    for j in range(9)
    for k in range(9)
]

# Constraint: 3x3 no repeating digit
distinct_3_by_3 = [
    Not(And(X[i][j][k], X[3*int(i/3)+a][3*int(j/3)+b][k]))
    for a in range(3)
    for b in range(3)
    for i in range(9)
    for j in range(9)
    for k in range(9)
    if not (i == 3*int(i/3)+a and j == 3*int(j/3)+b)
]


# Sudoku instance (0s are empty cells)
instance = (
    (0,1,0,4,0,2,0,5,0),
    (5,0,0,0,0,0,0,0,6),
    (0,0,0,3,0,1,0,0,0),
    (7,0,5,0,0,0,4,0,8),
    (0,0,0,0,0,0,0,0,0),
    (2,0,8,0,0,0,5,0,9),
    (0,0,0,9,0,6,0,0,0),
    (6,0,0,0,0,0,0,0,2),
    (0,7,0,1,0,3,0,4,0)
)

# Constraint: Must copy given non-zero numbers from instance
instance_c = [
  If(instance[i][j] == 0, True, X[i][j][instance[i][j]-1]) for i in range(9) for j in range(9)
]

s.add(instance_c + valid_cell + distinct_row + distinct_col + distinct_3_by_3)

# Function to construct 2-d array solution from a given model
def construct_solution(model):
  r = []
  for i in range(9):
    temp = []
    for j in range(9):
      for k in range(9):
        if(model.evaluate(X[i][j][k])):
          temp.append(k+1)
    r.append(temp)
  return r

if s.check() == sat:
    m = s.model()

    # Construct the solution grid
    r = construct_solution(m)

    # Print the solved Sudoku grid
    pp(r)
else:
  print("No Solution")


[[3, 1, 9, 4, 6, 2, 8, 5, 7],
 [5, 2, 4, 7, 9, 8, 1, 3, 6],
 [8, 6, 7, 3, 5, 1, 9, 2, 4],
 [7, 3, 5, 2, 1, 9, 4, 6, 8],
 [1, 9, 6, 8, 4, 5, 2, 7, 3],
 [2, 4, 8, 6, 3, 7, 5, 1, 9],
 [4, 5, 3, 9, 2, 6, 7, 8, 1],
 [6, 8, 1, 5, 7, 4, 3, 9, 2],
 [9, 7, 2, 1, 8, 3, 6, 4, 5]]


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(1 point)**

To check if the solution is unique or not, in our code we will negate the first solution that is found. What this will do is make the code try to find another solution other than the first. If after the negation of the first solution, the program returns unsolvable this means that there is only 1 unique solution. If the program outputs another solution this means that the solution is not unique.

In [ ]:
# s is the previous solution
new_s = Solver()
new_x = [[[
    Bool("cell_%s_%s_%s"%(i, j, k))
    for k in range(1, 10)]
    for j in range(9)]
    for i in range(9)]

# Constraint: Exclude previous solution (stored in r)
exclude_prev_solution = []
for i in range(9):
  for j in range(9):
    exclude_prev_solution.append(X[i][j][r[i][j]-1] == False)

new_s.add(instance_c + valid_cell + distinct_row + distinct_col + distinct_3_by_3 + exclude_prev_solution)
if(new_s.check() == unsat):
  print("No New Solution")
else:
    # Construct the solution grid
    r = construct_solution(m)

    # Print the solved Sudoku grid
    pp(r)

No New Solution


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be less than 5000. **(1 point)**

In [ ]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

# 9x9 matrix of Bool variables
X = [[[
    Bool(f"x_{i}_{j}_{k}")
    for k in range(1, 10)]
    for j in range(9)]
    for i in range(9)]

# Each cell contains a true bool
cells_c = [
    Or([X[i][j][k] for k in range(9)])
    for i in range(9) for j in range(9)]

# A cell contains 1 true boolean
one_per_cell_c = [
    Sum([If(X[i][j][k], 1, 0) for k in range(9)]) == 1
    for i in range(9) for j in range(9)]

# Each row contains a digit at most once
rows_c = [
    Or(Not(X[i][j][k]), Not(X[i][z][k]))
    for i in range(9)
    for k in range(9)
    for j in range(9)
    for z in range(j+1, 9)]

# Each column contains a digit at most once
cols_c = [
    Or(Not(X[i][j][k]), Not(X[z][j][k]))
    for j in range(9)
    for k in range(9)
    for i in range(9)
    for z in range(i+1, 9)]

# Each 3x3 square contains a digit at most once
sq_c = [
    Or(Not(X[3*i+m][3*j+n][k]), Not(X[3*i+p][3*j+q][k]))
    for i in range(3)
    for j in range(3)
    for k in range(9)
    for m in range(3) for n in range(3)
    for p in range(3) for q in range(3)
    if not (m == p and n == q)]

sudoku_c = cells_c + one_per_cell_c + rows_c + cols_c + sq_c

# Sudoku instance (0s are empty cells)
instance = [
    (0,0,0,4,0,2,0,5,0),
    (5,0,0,0,0,0,0,0,6),
    (0,0,0,3,0,1,0,0,0),
    (7,0,5,0,0,0,4,0,8),
    (0,0,0,0,0,0,0,0,0),
    (2,0,8,0,0,0,5,0,9),
    (0,0,0,9,0,6,0,0,0),
    (6,0,0,0,0,0,0,0,2),
    (0,7,0,1,0,3,0,4,0)
]

# instance constraints
instance_c = [
    If(instance[i][j] == 0,
       True,  # constraint for 0 (empty cell)
       X[i][j][instance[i][j] - 1])  # instance[i][j] is 1-based, list index is 0-based
    for i in range(9) for j in range(9)]

# Solve the puzzle
s = Solver()
s.add(sudoku_c + instance_c)
count = 0
while (s.check() == sat):
    count += 1
    m = s.model()
    r = [
        [sum([k+1 for k in range(9) if is_true(m.evaluate(X[i][j][k]))]) for j in range(9)]
        for i in range(9)
    ]

    negate_first_solution = Or([
        X[i][j][int(r[i][j])-1] == False
        for i in range(9)
        for j in range(9)
    ])
    s.add(negate_first_solution)

print("Total amount of solutions: ", count)



Total amount of solutions:  200


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(2 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
# Solution code for Miracle Sudoku using z3py
from z3 import *
miracle_x = [[[
    Bool("cell_%s_%s_%s"%(i, j, k))
    for k in range(1, 10)]
    for j in range(9)]
    for i in range(9)]

# Knight's constraint
knight_constraint = []
for i in range(9):
  for j in range(9):
    for k in range(9):
      if(i-2>=0 and j-1>=0): # check 2 up, 1 left
        knight_constraint.append(If(X[i][j][k] == True, X[i-2][j-1][k] == False, True))
      if(i-2>=0 and j+1<=8): # check 2 up, 1 right
        knight_constraint.append(If(X[i][j][k] == True, X[i-2][j+1][k] == False, True))
      if(i+2<=8 and j-1>=0): # check 2 down, 1 left
        knight_constraint.append(If(X[i][j][k] == True, X[i+2][j-1][k] == False, True))
      if(i+2<=8 and j+1<=8): # check 2 down, 1 right
        knight_constraint.append(If(X[i][j][k] == True, X[i+2][j+1][k] == False, True))
      if(i-1>=0 and j-2>=0): # check 1 up, 2 left
        knight_constraint.append(If(X[i][j][k] == True, X[i-1][j-2][k] == False, True))
      if(i-1>=0 and j+2<=8): # check 1 up, 2 right
        knight_constraint.append(If(X[i][j][k] == True, X[i-1][j+2][k] == False, True))
      if(i+1<=8 and j-2>=0): # check 1 down, 2 left
        knight_constraint.append(If(X[i][j][k] == True, X[i+1][j-2][k] == False, True))
      if(i+1<=8 and j+2<=8): # check 1 down, 2 right
        knight_constraint.append(If(X[i][j][k] == True, X[i+1][j+2][k] == False, True))

# King's Constraint, *Note: we only need to check for diagonal as normal sudoku rules covers adjacent squares
king_constraint = []
for i in range(9):
  for j in range(9):
    for k in range(9):
      if(i-1>=0 and j-1>=0): # check 1 up, 1 left
        king_constraint.append(If(X[i][j][k] == True, X[i-1][j-1][k] == False, True))
      if(i-1>=0 and j+1<=8): # check 2 up, 1 right
        king_constraint.append(If(X[i][j][k] == True, X[i-1][j+1][k] == False, True))
      if(i+1<=8 and j-1>=0): # check 2 down, 1 left
        king_constraint.append(If(X[i][j][k] == True, X[i+1][j-1][k] == False, True))
      if(i+1<=8 and j+1<=8): # check 2 down, 1 right
        king_constraint.append(If(X[i][j][k] == True, X[i+1][j+1][k] == False, True))

orthogonal_constraint = []
for i in range(9):
  for j in range(9):
    for k in range(9):
      if(k+1<=8 and i-1>=0): # check 1 up, if up = current + 1
        king_constraint.append(If(X[i][j][k] == True, X[i-1][j][k+1] == False, True))
      if(k-1>=0 and i-1>=0): # check 1 up, if up = current - 1
        king_constraint.append(If(X[i][j][k] == True, X[i-1][j][k-1] == False, True))
      if(k+1<=8 and i+1<=8): # check 1 down, if up = current + 1
        king_constraint.append(If(X[i][j][k] == True, X[i+1][j][k+1] == False, True))
      if(k-1>=0 and i+1<=8): # check 1 down, if up = current - 1
        king_constraint.append(If(X[i][j][k] == True, X[i+1][j][k] == False, True))
      if(k+1<=8 and j-1<=8): # check 1 left, if left = current + 1
        king_constraint.append(If(X[i][j][k] == True, X[i][j-1][k+1] == False, True))
      if(k-1>=0 and j-1<=8): # check 1 left, if left = current - 1
        king_constraint.append(If(X[i][j][k] == True, X[i][j-1][k-1] == False, True))
      if(k+1<=8 and j+1<=8): # check 1 right, if right = current + 1
        king_constraint.append(If(X[i][j][k] == True, X[i][j+1][k+1] == False, True))
      if(k-1>=0 and j+1<=8): # check 1 right, 1 right
        king_constraint.append(If(X[i][j][k] == True, X[i][j+1][k-1] == False, True))

# Sudoku instance (0s are empty cells)
instance = (
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,1,0,0,0,0,0,0),
    (0,0,0,0,0,0,2,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0)
)

# instance constraints
instance_miracle_constraint = [
    If(instance[i][j] == 0,
       True,  # constraint for 0 (empty cell)
       X[i][j][instance[i][j] - 1])  # instance[i][j] is 1-based, list index is 0-based
    for i in range(9) for j in range(9)]


miracle_s = Solver()
miracle_s.add(valid_cell + distinct_row + distinct_col + distinct_3_by_3 + knight_constraint + king_constraint + orthogonal_constraint + instance_miracle_constraint)
if(miracle_s.check() == unsat):
  print("No New Solution")
else:
    miracle_m = miracle_s.model()
    # Construct the solution grid
    r = construct_solution(miracle_m)

    # Print the solved Sudoku grid
    pp(r)

[[4, 8, 3, 7, 2, 6, 1, 5, 9],
 [7, 2, 6, 1, 5, 9, 4, 8, 3],
 [1, 5, 9, 4, 8, 3, 7, 2, 6],
 [8, 3, 7, 2, 6, 1, 5, 9, 4],
 [2, 6, 1, 5, 9, 4, 8, 3, 7],
 [5, 9, 4, 8, 3, 7, 2, 6, 1],
 [3, 7, 2, 6, 1, 5, 9, 4, 8],
 [6, 1, 5, 9, 4, 8, 3, 7, 2],
 [9, 4, 8, 3, 7, 2, 6, 1, 5]]




---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(1 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

Table only including when r is false to avoid it from being too large

Entailment 1

| p | q | r | s | t | ~p | ~s | t->q | s->~p | p->q | (p->q)->r |
|---|---|---|---|---|----|----|------|-------|------|-----------|
| T | T | F | T | T |  F | F  |  T   |   F   |  T   |    F      |
| T | T | F | T | F |  F | F  |  T   |   F   |  T   |    F      |
| T | T | F | F | T |  F | T  |  T   |   T   |  T   |    F      |
| T | T | F | F | F |  F | T  |  T   |   T   |  T   |    F      |
| T | F | F | T | T |  F | F  |  F   |   F   |  F   |    T      |
| T | F | F | T | F |  F | F  |  T   |   F   |  F   |    T      |
| T | F | F | F | T |  F | T  |  F   |   T   |  F   |    T      |
| T | F | F | F | F |  F | T  |  T   |   T   |  F   |    T      |
| F | T | F | T | T |  T | F  |  T   |   T   |  T   |    F      |
| F | T | F | T | F |  T | F  |  T   |   T   |  T   |    F      |
| F | T | F | F | T |  T | T  |  T   |   T   |  T   |    F      |
| F | T | F | F | F |  T | T  |  T   |   T   |  T   |    F      |
| F | F | F | T | T |  T | F  |  F   |   T   |  T   |    F      |
| F | F | F | T | F |  T | F  |  T   |   T   |  T   |    F      |
| F | F | F | F | T |  T | T  |  F   |   T   |  T   |    F      |
| F | F | F | F | F |  T | T  |  T   |   T   |  T   |    F      |  

Entailment 2

| p | q | (q->p) | p->(q->p) | p->(p->(q->p)) | q->(p->(p->(q->p))) |
|---|---|--------|-----------|----------------|---------------------|
| T | T |   T    |     T     |       T        |          T          |
| T | F |   T    |     T     |       T        |          T          |
| F | T |   F    |     T     |       T        |          T          |
| F | F |   T    |     T     |       T        |          T          |

Since the right hand side is always true, it doesn't matter what the left hand
expression is.


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(1 point)**

In [ ]:
# Create Boolean Constants
p = Bool('p')
q = Bool('q')
r = Bool('r')
s = Bool('s')
t = Bool('t')

# Encode the constraints for the first part
constraints_1 = [
    Implies(Implies(p, q), r),
    Implies(s, Not(p)),
    t,
    Not(s),
    Implies(t, q)
]

# Expression for first part
expression_1 = And(constraints_1)

# Function to prove first entailment
def prove_1(f):
  s = Solver()
  s.add((If(r == False, f == True, True)) == False)
  if s.check() == sat:
      return "proved"
  else:
      return "not proved"


# Encode the constraints for the second part
constraints_2 = [
    Implies(q, Implies(p, Implies(p, Implies(q, p))))
]

# Expression for second part
expression_2 = And(constraints_2)

# Function to prove second entailment
def prove_2(f):
  s = Solver()
  s.add(If(And(p == True, q == True), expression_2, True))
  s.add(If(And(p == True, q == False), expression_2, True))
  s.add(If(And(p == False, q == True), expression_2, True))
  s.add(If(And(p == False, q == False), expression_2, True))
  if s.check() == sat:
      return "proved"
  else:
      return "not proved"

print("Expression 1 is", prove_1(expression_1))
print("Expression 2 is", prove_2(expression_2))

Expression 1 is proved
Expression 2 is proved


In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test

In [ ]:
# test